In [1]:
import pandas as pd
from functools import reduce

In [2]:
# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# Counts the number of singular players 
total_players=purchase_data["SN"].nunique()

#Creates a DataFrame for presenting the results
player_count=pd.DataFrame([total_players],columns=["Total Players"])

#Displays the resuts DataFrame
player_count

,Total Players
0,576


In [4]:
# Calculates the total revenue by summing the price column
total_revenue=purchase_data["Price"].sum()

# Calculates the number of purchases by counting the number of Item ID in the csv
purchases_number=purchase_data["Item ID"].count()

# Calculates the unique number of Items by Item ID
unique_items=purchase_data["Item ID"].nunique()

# Cleans the data for duplicates in the Item ID column, this way we only have singular items
clean_data=purchase_data.drop_duplicates(subset ="Item ID", keep = 'last',inplace=False)

# Calculates the average price per item
average_price=clean_data['Price'].mean()

# Creates a new DataFrame with the calculated information
total_purchasing_analysis=pd.DataFrame([[unique_items,average_price,purchases_number,total_revenue]],
                                 columns=["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"])

# Changes the currency format
total_purchasing_analysis['Total Revenue']= total_purchasing_analysis['Total Revenue'].map('${:,.2f}'.format)
total_purchasing_analysis['Average Price']= total_purchasing_analysis['Average Price'].map('${:,.2f}'.format)

# Displays the results DataFrame
total_purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.04,780,"$2,379.77"


In [5]:
# Counts the number of unique players in the database
total_players=purchase_data["SN"].nunique()

# Cleans the database for duplicate players
clean_data=purchase_data.drop_duplicates(subset ="SN", keep = 'last',inplace=False)

# Counts and groups the data by gender, then sorts the values descending
grouped_data=clean_data.groupby(['Gender']).count().sort_values(['Purchase ID'],ascending=False)

# Calculates the percentaje of players and adds a column for the results
grouped_data['Percentaje of Players']=grouped_data['Purchase ID']/total_players*100

# Renames of the columns to match the new data and eliminates columns with repeated information
final_data=grouped_data.rename(columns={'Purchase ID':'Total Count'}).drop(['SN','Age','Item ID','Item Name','Price'],axis=1)

# Changes the percentaje format
final_data['Percentaje of Players']= final_data['Percentaje of Players'].map('%{:,.2f}'.format)

# Displays the results DataFrame
final_data

,Total Count,Percentaje of Players
Gender,,
Male,484,%84.03
Female,81,%14.06
Other / Non-Disclosed,11,%1.91


In [6]:
# Groups the purchases by gender and counts the number of purchases made by each gender, 
# renames the column for a better understanding and drops the other columns
purchase_count=purchase_data.groupby(['Gender']).count().rename(columns={'Purchase ID':'Purchase Count'}).drop(['SN','Age','Item ID','Item Name','Price'],axis=1)

# Groups the purchases by gender and summs the price of purchases made by each gender
# renames the column for a better understanding and drops the other columns
total_purchase=purchase_data.groupby(['Gender']).sum().rename(columns={'Price':'Total Purchase Value'}).drop(['Purchase ID','Age','Item ID'],axis=1)

# Groups the purchases by gender and calculates de price the price of purchases made by each gender
# renames the column for a better understanding and drops the other columns
avg_purchase_price=purchase_data.groupby(['Gender']).mean().rename(columns={'Price':'Average Purchase Price'}).drop(['Purchase ID','Age','Item ID'],axis=1)

# Calculates the total pruchase per player and stores it in a data frame
# renames the column for a better understanding and drops the other columns
total_purchase_per_person=purchase_data.groupby(['SN']).sum().rename(columns={'Price':'Total Purchase Per Person'}).drop(['Purchase ID','Age','Item ID'],axis=1)

# Merges the previous dataframe with a copy of the original dataframe that has been cleaned for duplicates of the players
# for the merging the index of the cleaned data is set to be the same as the previous dataframe
merged_data=purchase_data.drop_duplicates(subset ='SN', keep = 'last',inplace=False).set_index('SN').merge(total_purchase_per_person,right_index=True,left_index=True)

# Groups the players by gender and calculates the average for the other columns
# renames the column for a better understanding and drops the other columns
avg_purchase_per_person=merged_data.groupby('Gender').mean().rename(columns={'Total Purchase Per Person':'Average Purchase Per Person'}).drop(['Purchase ID','Age','Item ID','Price'],axis=1)

# Applies the lambda function for merging all of the dataframes together by their index (in this case the Gender)
gender_purchase_analysis=reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True), [purchase_count,avg_purchase_price,total_purchase,avg_purchase_per_person])

# Changes the currency format
gender_purchase_analysis['Total Purchase Value']= gender_purchase_analysis['Total Purchase Value'].map('${:,.2f}'.format)
gender_purchase_analysis['Average Purchase Price']= gender_purchase_analysis['Average Purchase Price'].map('${:,.2f}'.format)
gender_purchase_analysis['Average Purchase Per Person']= gender_purchase_analysis['Average Purchase Per Person'].map('${:,.2f}'.format)

# Displays the results dataframe
gender_purchase_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
# Groups the purchases by player and counts the number of purchases made by each player, 
# renames the column for a better understanding and drops the other columns
purchase_count=purchase_data.groupby(['SN']).count().rename(columns={'Purchase ID':'Purchase Count'}).drop(['Gender','Age','Item ID','Item Name','Price'],axis=1)

# Groups the purchases by player and calculates de price the price of purchases made by each player
# renames the column for a better understanding and drops the other columns
avg_purchase_price=purchase_data.groupby(['SN']).mean().rename(columns={'Price':'Average Purchase Price'}).drop(['Purchase ID','Age','Item ID'],axis=1)

# Calculates the total pruchase per player and stores it in a data frame
# renames the column for a better understanding and drops the other columns
total_purchase_per_person=purchase_data.groupby(['SN']).sum().rename(columns={'Price':'Total Purchase Value'}).drop(['Purchase ID','Age','Item ID'],axis=1)

# Applies the lambda function for merging all of the dataframes together by their index (in this case the Player)
spenders=reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True), [purchase_count,avg_purchase_price,total_purchase_per_person])

# Changes the currency format
spenders['Total Purchase Value']= spenders['Total Purchase Value'].map('${:,.2f}'.format)
spenders['Average Purchase Price']= spenders['Average Purchase Price'].map('${:,.2f}'.format)

# Displays the results dataframe
spenders.sort_values(['Total Purchase Value'],ascending=False).head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Haillyrgue51,3,$3.17,$9.50
Phistym51,2,$4.75,$9.50
Lamil79,2,$4.64,$9.29
Aina42,3,$3.07,$9.22
Saesrideu94,2,$4.59,$9.18


In [8]:
# Groups the purchases by item and counts the number of purchases made for each item, 
# renames the column for a better understanding and drops the other columns
purchase_count=purchase_data.groupby(['Item ID']).count().rename(columns={'Purchase ID':'Purchase Count'}).drop(['Gender','Age','SN','Item Name','Price'],axis=1)

# Calculates the total pruchase per item and stores it in a data frame
# renames the column for a better understanding and drops the other columns
total_purchase_per_item=purchase_data.groupby(['Item ID']).sum().rename(columns={'Price':'Total Purchase Value'}).drop(['Purchase ID','Age'],axis=1)

# Applies the lambda function for merging all of the dataframes together by their index (in this case the Player)
profitable_items=reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True), [purchase_count,purchase_data.drop_duplicates(subset ='Item ID', keep = 'last',inplace=False).set_index('Item ID'),total_purchase_per_item])

# Sets the desired indexes, drops the unused information columns and sorts the values by the total purchase value
final_data=profitable_items.reset_index().set_index(['Item ID','Item Name']).drop(['Purchase ID','SN','Age','Gender'],axis=1).sort_values(['Total Purchase Value'],ascending=False).head()

# Changes the percentaje format
final_data['Price']= final_data['Price'].map('${:,.2f}'.format)
final_data['Total Purchase Value']= final_data['Total Purchase Value'].map('${:,.2f}'.format)

# Displays the results DataFrame
final_data

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


In [9]:
# Groups the purchases by item and counts the number of purchases made for each item, 
# renames the column for a better understanding and drops the other columns
purchase_count=purchase_data.groupby(['Item ID']).count().rename(columns={'Purchase ID':'Purchase Count'}).drop(['Gender','Age','SN','Item Name','Price'],axis=1)

# Calculates the total pruchase per item and stores it in a data frame
# renames the column for a better understanding and drops the other columns
total_purchase_per_item=purchase_data.groupby(['Item ID']).sum().rename(columns={'Price':'Total Purchase Value'}).drop(['Purchase ID','Age'],axis=1)

# Applies the lambda function for merging all of the dataframes together by their index (in this case the Player)
popular_items=reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True), [purchase_count,purchase_data.drop_duplicates(subset ='Item ID', keep = 'last',inplace=False).set_index('Item ID'),total_purchase_per_item])

# Sets the desired indexes, drops the unused information columns and sorts the values by the total purchase count
final_data=popular_items.reset_index().set_index(['Item ID','Item Name']).drop(['Purchase ID','SN','Age','Gender'],axis=1).sort_values(['Purchase Count'],ascending=False).head()

# Changes the percentaje format
final_data['Price']= final_data['Price'].map('${:,.2f}'.format)
final_data['Total Purchase Value']= final_data['Total Purchase Value'].map('${:,.2f}'.format)

# Displays the results DataFrame
final_data

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [10]:
# Counts the number of singular players 
total_players=purchase_data["SN"].nunique()

#Create the values for the binning
bins=[0,9,14,19,24,29,34,39,999]

# Create the labels for the bins
group_labels=['< 10','10-14','15-19','20-24','25-29','30-34','35-39','40 +']

# Creates dataframe with the unique players
age_demographics= purchase_data.drop_duplicates(subset ='SN', keep = 'last',inplace=False)

# Creates a column that assigns a bin to every player
age_demographics["Age Ranges"] = pd.cut(age_demographics["Age"], bins, labels=group_labels)

# Creates a new dataframe agrouped by the age range (bins) and drops the unuseful columns
age_demographics=age_demographics.groupby(['Age Ranges']).count().rename(columns={'Price':'Total Count'}).drop(['Purchase ID','SN','Age','Gender','Item ID','Item Name'],axis=1)

# Creates a new column with the percentage of players (player per age range divided by the total players)
age_demographics['Percentaje of Players']=age_demographics['Total Count']/total_players*100

# Changes the percentaje format
age_demographics['Percentaje of Players']= age_demographics['Percentaje of Players'].map('%{:,.2f}'.format)

# Displays the results DataFrame
age_demographics

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Total Count,Percentaje of Players
Age Ranges,,
< 10,17,%2.95
10-14,22,%3.82
15-19,107,%18.58
20-24,258,%44.79
25-29,77,%13.37
30-34,52,%9.03
35-39,31,%5.38
40 +,12,%2.08


In [11]:
#Create the values for the binning
bins=[0,9,14,19,24,29,34,39,999]

# Create the labels for the bins
group_labels=['< 10','10-14','15-19','20-24','25-29','30-34','35-39','40 +']

# Creates dataframe with the same information as the original for making calculations
new_data= purchase_data

# Creates a column that assigns a bin to every player
new_data["Age Ranges"] = pd.cut(new_data["Age"], bins, labels=group_labels)

# Creates a new dataframe agrouped by the age range (bins), counts the number of purchases per bin 
# renames the useful information for better understanding and drops the unuseful columns
purchase_count=new_data.groupby(['Age Ranges']).count().rename(columns={'Price':'Purchase Count'}).drop(['Purchase ID','SN','Age','Gender','Item ID','Item Name'],axis=1)

# Creates a new dataframe agrouped by the age range (bins), summs the purchase price per bin 
# renames the useful information for better understanding and drops the unuseful columns
total_purchase_value=new_data.groupby(['Age Ranges']).sum().rename(columns={'Price':'Total Purchase Value'}).drop(['Purchase ID','Age','Item ID'],axis=1)

# Creates a new dataframe agrouped by the age range (bins), calculates the average of the purchase price per bin 
# renames the useful information for better understanding and drops the unuseful columns
avg_purchase_price=new_data.groupby(['Age Ranges']).mean().rename(columns={'Price':'Average Purchase Price'}).drop(['Purchase ID','Age','Item ID'],axis=1)

# Creates a new dataframe agrouped by the name of the player, summs the price of purchase per player
# renames the useful information for better understanding and drops the unuseful columns
avg_total_purchase_per_person=new_data.groupby(['SN']).sum().rename(columns={'Price':'Average Total Purchase per Person'}).drop(['Purchase ID','Age','Item ID'],axis=1)

# Merges the previous dataframe with the dataframe that has the Age Range labels assignes to each player 
# (before merging is necesary to eliminate duplicates in the second dataframe and match the index to the player name)
# groups the information per bins("Age Ranges") by calculating the mean of the total price per player
# and drops the unuseful columns
avg_total_purchase_per_person=avg_total_purchase_per_person.merge(new_data.drop_duplicates(subset ='SN', keep = 'last',inplace=False).set_index('SN'),right_index=True,left_index=True).groupby(['Age Ranges']).mean().drop(['Purchase ID','Age','Item ID','Price'],axis=1)

# Applies the lambda function for merging all of the dataframes together by their index (in this case the Player)
age_purchase_analysis=reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True), [purchase_count,avg_purchase_price,total_purchase_value,avg_total_purchase_per_person])

# Changes the currency format
age_purchase_analysis['Total Purchase Value']= age_purchase_analysis['Total Purchase Value'].map('${:,.2f}'.format)
age_purchase_analysis['Average Purchase Price']= age_purchase_analysis['Average Purchase Price'].map('${:,.2f}'.format)
age_purchase_analysis['Average Total Purchase per Person']= age_purchase_analysis['Average Total Purchase per Person'].map('${:,.2f}'.format)


#Displays the results dataframe
age_purchase_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Ranges,,,,
< 10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40 +,13,$2.94,$38.24,$3.19
